# Parker Solar Probe Data Download Example

Tamar Ervin

- Downloading [PSP](https://link.springer.com/article/10.1007/s11214-015-0211-6) data with [PySPEDAS](https://pyspedas.readthedocs.io/en/latest/)
- Using [pyTplot](https://pytplot.readthedocs.io/en/latest/) to plot the data


In [ ]:

import pyspedas
from pyspedas import time_string
from pytplot import tplot, get_data
import astrospice
import sunpy 
import sunpy.coordinates as scoords
import sys, os
import datetime
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt

for sc in ['psp','solar orbiter'] : kernels = astrospice.registry.get_kernels(sc,'predict') 


## Download and Plot the Data

Example data to download:
- [FIELDS](https://link.springer.com/article/10.1007/s11214-016-0244-5): Radial, Tangential, Normal (RTN) magnetic field data
- [SWEAP/SPAN-I](https://iopscience.iop.org/article/10.3847/1538-4357/ac93f5) Proton: Radial, Tangential, Normal (RTN) proton velocity and proton density data
- [SWEAP/SPAN-I](https://iopscience.iop.org/article/10.3847/1538-4357/ac93f5) Alpha Particle: Radial, Tangential, Normal (RTN) alpha particle velocity and density

Data will download to a folder titled 'psp_data' in this same repo!

Don't worry if this takes a while to run! The data is at a very high cadence and takes a bit to download depending on the time range

## Plot the data using pyTplot

[pyTplot](https://pytplot.readthedocs.io/en/latest/) is a Python package that works with PySPEDAS to plot space physics data! It already has all the information needed to plot observables in terms of their units! You can also create your own plots of the data using matplotlib.


In [ ]:
### ------- TIME PERIOD OF INTEREST ------- ###
# this is an example of data from the heliospheric current sheet (HCS) crossing during PSP Encounter 15
time_range = ['2023-03-17/12:00', '2023-03-18/12:00']

In [ ]:
### ------- FIELDS: MAG RTN DATA ------- ###
fields_vars = pyspedas.psp.fields(trange=time_range, datatype='mag_RTN_4_Sa_per_Cyc')

### print out the variables stored in the magnetic field data
print(fields_vars)

### get the RTN magnetic field
B_RTN = get_data('psp_fld_l2_mag_RTN_4_Sa_per_Cyc')

### plot the data!
tplot(['psp_fld_l2_mag_RTN_4_Sa_per_Cyc'])

### CONVERT TIME FROM JULIAN TIME TO DATETIME OBJECT
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in B_RTN.times]

### CREATE DATAFRAME
rd = {'Time': date_obj, 'Br': B_RTN.y[:, 0], 'Bt': B_RTN.y[:, 1], 'Bn': B_RTN.y[:, 2]}
fields = pd.DataFrame(data=rd)

### SAVE DATAFRAME AS CSV
fields.to_csv(os.path.join('results', 'fields.csv')) 

In [ ]:
### ------- SPAN-I: PROTON (HYDROGEN) MOMENTS ------- ###
### download proton data
proton_vars = pyspedas.psp.spi(trange=time_range, datatype='sf00_l3_mom', level='l3')

### print out the variables stored in the proton data
print(proton_vars)

### get the RTN velocity, density, and temperature
Np = get_data('psp_spi_DENS')
vp_RTN = get_data('psp_spi_VEL_RTN_SUN')
Tp = get_data('psp_spi_TEMP')

### CONVERT TIME FROM JULIAN TIME TO DATETIME OBJECT
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in Np.times]

### CREATE DATAFRAME
rd = {'Time': date_obj, 'vr': vp_RTN.y[:, 0], 'vt': vp_RTN.y[:, 1], 'vn': vp_RTN.y[:, 2], 'Np': Np.y, 'Tp': Tp.y}
protons = pd.DataFrame(data=rd)

### SAVE DATAFRAME AS CSV
protons.to_csv(os.path.join('results', 'protons.csv'))

### plot the data!
tplot(['psp_spi_DENS', 'psp_spi_VEL_RTN_SUN', 'psp_spi_TEMP'])

In [ ]:
### ------- SPAN-I: ALPHA PARTICLE (HELIUM) MOMENTS ------- ###
### download alpha particle data
alpha_vars = pyspedas.psp.spi(trange=time_range, datatype='sf0a_l3_mom', level='l3')

### print out the variables stored in the alpha particle data
print(alpha_vars)

### READ IN SWEAP VELOCITY (RTN), DENSITY, AND TEMPERATURE DATA
Na = get_data('psp_spi_DENS')
va_RTN = get_data('psp_spi_VEL_RTN_SUN')
Ta = get_data('psp_spi_TEMP')

### CONVERT TIME FROM JULIAN TIME TO DATETIME OBJECT
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in Na.times]

### CREATE DATAFRAME
rd = {'Time': date_obj, 'vr': va_RTN.y[:, 0], 'vt': va_RTN.y[:, 1], 'vn': va_RTN.y[:, 2], 'Na': Na.y, 'Ta': Ta.y}
alphas = pd.DataFrame(data=rd)

### SAVE DATAFRAME AS CSV
alphas.to_csv(os.path.join('results', 'alphas.csv'))

### plot the data!
tplot(['psp_spi_DENS', 'psp_spi_VEL_RTN_SUN', 'psp_spi_TEMP'])

# Create Parker FIELDS/SWEAP Dataframe

Now that we have a dataframe for each instrument, we then use pandas to merge the dataframes as a function of time.



In [ ]:
### merge the PAS and MAG dataframes
merged_df = pd.merge_asof(fields, protons, on='Time', direction='backward')

### merge the HIS and newly merged dataframe
merged_df = pd.merge_asof(alphas, merged_df, on='Time', direction='backward')
merged_df = merged_df.set_index('Time')

## Find the spacecraft trajectory

Now we will use astrospice to generate the trajectory of the spacecraft. We transform from inertial coordinates to the solar co-rotating (Carrington) frame!

In [ ]:
### Create SkyCoord for Parker in the inertial (J2000) frame
psp_inertial = astrospice.generate_coords(
    'SOLAR PROBE PLUS', pd.to_datetime(merged_df.index.to_list())

)

### Transform to solar co-rotating frame 
psp_carrington = psp_inertial.transform_to(
    scoords.HeliographicCarrington(observer="self")
)

Now we will add the position information to our merged dataframe and save as a CSV file!

In [ ]:
### ADD POSITION INFORMAITON AND SAVE
parker = merged_df.copy()
parker['lon'] = psp_carrington.lon.value
parker['lat'] = psp_carrington.lat.value
parker['rAU'] = psp_carrington.radius.to(u.AU).value
parker['NpR2'] = parker.Np * (parker.rAU ** 2)
parker['BrR2'] = parker.Br * (parker.rAU ** 2)
parker.to_csv(os.path.join('results', 'parker.csv'))
parker